## Setup and imports
In this section we shall install the various dependencies that are required to run the analyzer

In [2]:
%%capture
%pip install matplotlib

In [ ]:
import json
import matplotlib.pyplot as plt